#**Лабораторная работа № 3 "Построение гибридной криптосистемы"**
**В рамках курса "Основы информационной безопасности"**  
*Специальность "Информационная безопасность автоматизированных систем", 2 курс, 2 семестр*  

Результат данной лабораторной работы - [гибридная криптосистема](https://www.youtube.com/watch?v=VPvZbMXfv_0) - будет сочетать в себе преимущества [симметричных](http://mf.grsu.by/UchProc/livak/b_protect/zok_2.htm) и [асимметричных](https://bstudy.net/905249/tehnika/asimmetrichnye_shifry) алгоритмов шифрования и в целом даже напоминать системы, использующиеся на практике. Симметричные алгоритмы работают быстрее ассиметричных и ими удобно шифровать большие объемы информации. Зато асимметричные алгоритмы не требуют наличия секретного канала для передачи ключа.   
Работать она будет следующим образом: сообщение шифруется при помощи симметричного алгоритма, а ключ симметричного алгоритма - ассиметричным. В качестве асимметричного алгоритма будет использоваться [RSA](http://www.michurin.net/computer-science/rsa.html), симметричный алгорим выбирается в соответствии с вариантом. Варианты приведены ниже (все шифры блочные).



##Задание  
Реализовать приложение (можно консольное), которое будет выполнять три указанных ниже сценарии: 

###1. Генерация ключей гибридной системы
*Входные параметры:*  
*1) путь, по которому сериализовать зашифрованный симметричный ключ;*  
*2) путь, по которому сериализовать открытый ключ;*  
*3) путь, по которому сериазизовать закрытый ключ.*

1.1. Сгеренировать ключ для симметричного алгоритма.  
1.2. Сгенерировать ключи для ассиметричного алгоритма.  
1.3. Сериализовать ассиметричные ключи.   
1.4. Зашифровать ключ симметричного шифрования открытым ключом и сохранить по указанному пути. 

###2. Шифрование данных гибридной системой
*Входные параметры:*  
*1) путь к шифруемому текстовому файлу (очевидно, что файл должен быть достаточно объемным);*  
*2) путь к закрытому ключу ассиметричного алгоритма;*  
*3) путь к зашированному ключу симметричного алгоритма;*  
*4) путь, по которому сохранить зашифрованный текстовый файл;*  

2.1. Расшифровать симметричный ключ.  
2.2. Зашифровать текст симметричным алгоритмом и сохранить по указанному пути.   

###3. Дешифрование данных гибридной системой
*Входные парметры:*  
*1) путь к зашифрованному текстовому файлу;*  
*2) путь к закрытому ключу ассиметричного алгоритма;*  
*3) путь к зашированному ключу симметричного алгоритма;*  
*4) путь, по которому сохранить расшифрованный текстовый файл.*  

3.1. Расшифровать симметричный ключ.  
3.2. Расшифровать текст симметричным алгоритмом и сохранить по указанному пути. 

##Небольшие ремарки к реализации приложения
Я очень расчитываю не увидеть в вашем программном обеспечении захардкоженных входных параметров. Вы можете как считать их с клавиатуры, так и из файла. Ниже приведен пример работы с json.  

Также я расчитываю, что ваше приложение будет сигнализировать о выполнении различных этапов своей работы.

А еще, что оно не будет представлять из себя сплошную простыню кода без единой процедуры.

Также было бы очень приятно увидеть лабораторную работу в виде грамотно оформленного репозитория на [github](https://github.com/)/[bitbucket](https://bitbucket.org/)/так далее.

In [2]:
settings= {
    'initial_file':'path/to/inital/file.txt',
    'encrypted_file':'path/to/encrypted/file.txt',
    'decrypted_file':'path/to/decrypted/file.txt',
    'symmetric_key':'path/to/symmetric/key.txt',
    'public_key':'path/to/public/key.pem',
    'secret_key':'path/to/secret/key.pem',
}


In [3]:
import json
# пишем в файл
with open('settings.json', 'w') as fp:
    json.dump(settings, fp)
# читаем из файла
with open('settings.json') as json_file:
    json_data = json.load(json_file)

print(json_data)

{'initial_file': 'path/to/inital/file.txt', 'encrypted_file': 'path/to/encrypted/file.txt', 'decrypted_file': 'path/to/decrypted/file.txt', 'symmetric_key': 'path/to/symmetric/key.txt', 'public_key': 'path/to/public/key.pem', 'secret_key': 'path/to/secret/key.pem'}


Чтобы у пользователя была возможность выбора одного из описанных выше режимов работы, можно воспользоваться argparse и организовать группу взаимоисключающих параметров командной строки.

##Работа с библиотекой cryptography
В данном разделе представлены примеры работы с библиотекой [cryptography](https://cryptography.io/en/latest/), которые могут вам пригодиться при выполнении данной лабораторной работы. Сама библиотека прекрасно задокументирована, так что, если вы владете английским языком, советую не проходить мимо. Если вы ко второму (третьему) курсу не владеете английским языком в мере, позволяющей вам читать технические тексты, то соболезную.

In [4]:
pip install cryptography

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [18]:
# генерация ключа симметричного алгоритма шифрования
import os #можно обойтись стандартным модулем

key = os.urandom(32) # это байты

print(type(key))
print(key)

<class 'bytes'>
b'"=3>\xb0\xaf\xc6\x14\xbb\x12\xbf\x18\xf6\xa2\xacGBr\x8c\x90\xd1\x8a\xba@pv\xee8\xfa~e+'


In [19]:
# сериализация ключа симмеричного алгоритма в файл
file_name = 'symmetric.txt'
with open(file_name, 'wb') as key_file:
  key_file.write(key)

In [20]:
# десериализация ключа симметричного алгоритма
with open(file_name, mode='rb') as key_file: 
    content = key_file.read()

print(type(content))
print(content)

<class 'bytes'>
b'"=3>\xb0\xaf\xc6\x14\xbb\x12\xbf\x18\xf6\xa2\xacGBr\x8c\x90\xd1\x8a\xba@pv\xee8\xfa~e+'


In [21]:
# паддинг данных для работы блочного шифра - делаем длину сообщения кратной длине шифркуемого блока
from cryptography.hazmat.primitives import padding

padder = padding.ANSIX923(32).padder()
text = bytes('кто прочитал тот здохнет', 'UTF-8')
padded_text = padder.update(text)+padder.finalize()

print(text)
print(padded_text)

b'\xd0\xba\xd1\x82\xd0\xbe \xd0\xbf\xd1\x80\xd0\xbe\xd1\x87\xd0\xb8\xd1\x82\xd0\xb0\xd0\xbb \xd1\x82\xd0\xbe\xd1\x82 \xd0\xb7\xd0\xb4\xd0\xbe\xd1\x85\xd0\xbd\xd0\xb5\xd1\x82'
b'\xd0\xba\xd1\x82\xd0\xbe \xd0\xbf\xd1\x80\xd0\xbe\xd1\x87\xd0\xb8\xd1\x82\xd0\xb0\xd0\xbb \xd1\x82\xd0\xbe\xd1\x82 \xd0\xb7\xd0\xb4\xd0\xbe\xd1\x85\xd0\xbd\xd0\xb5\xd1\x82\x00\x00\x03'


In [22]:
# шифрование текста симметричным алгоритмом
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes

iv = os.urandom(16) #случайное значение для инициализации блочного режима, должно быть размером с блок и каждый раз новым
cipher = Cipher(algorithms.AES(key), modes.CBC(iv))
encryptor = cipher.encryptor()
c_text = encryptor.update(padded_text) + encryptor.finalize()

print(c_text)

b"LG\x14\xdc\xf9C\x94\x83u\xf7\xa6;1\xe6'\x15\x9a5\x87Bl\x04C\xb4\x9f\xe9\x86\xa7\x05\xa1\x99|XH&\xd7r;\x85\xcb\xbaPG<d\x91\xcca"


In [23]:
# дешифрование и депаддинг текста симметричным алгоритмом

decryptor = cipher.decryptor()
dc_text = decryptor.update(c_text) + decryptor.finalize()

unpadder = padding.ANSIX923(32).unpadder()
unpadded_dc_text = unpadder.update(dc_text) + unpadder.finalize()

print(dc_text.decode('UTF-8'))
print(unpadded_dc_text.decode('UTF-8'))

кто прочитал тот здохнет  
кто прочитал тот здохнет


In [24]:
# генерация пары ключей для асимметричного алгоритма шифрования
from cryptography.hazmat.primitives.asymmetric import rsa
from cryptography.hazmat.primitives import serialization

keys = rsa.generate_private_key(
    public_exponent=65537,
    key_size=2048
)
private_key = keys
public_key = keys.public_key()

print(type(private_key))
print(private_key)
print(type(public_key))
print(public_key)

<class 'cryptography.hazmat.backends.openssl.rsa._RSAPrivateKey'>
<class 'cryptography.hazmat.backends.openssl.rsa._RSAPublicKey'>


In [25]:
# сериализация открытого ключа в файл
public_pem = 'public.pem'
with open(public_pem, 'wb') as public_out:
        public_out.write(public_key.public_bytes(encoding=serialization.Encoding.PEM,
             format=serialization.PublicFormat.SubjectPublicKeyInfo))

# сериализация закрытого ключа в файл
private_pem = 'private.pem'
with open(private_pem, 'wb') as private_out:
        private_out.write(private_key.private_bytes(encoding=serialization.Encoding.PEM,
              format=serialization.PrivateFormat.TraditionalOpenSSL,
              encryption_algorithm=serialization.NoEncryption()))

In [26]:
from cryptography.hazmat.primitives.serialization import load_pem_public_key, load_pem_private_key

# десериализация открытого ключа
with open(public_pem, 'rb') as pem_in:
  public_bytes = pem_in.read()
d_public_key = load_pem_public_key(public_bytes)
# десериализация закрытого ключа
with open(private_pem, 'rb') as pem_in:
  private_bytes = pem_in.read()
d_private_key = load_pem_private_key(private_bytes,password=None,)

print(type(d_private_key))
print(d_private_key)
print(type(d_public_key))
print(d_public_key)

<class 'cryptography.hazmat.backends.openssl.rsa._RSAPrivateKey'>
<class 'cryptography.hazmat.backends.openssl.rsa._RSAPublicKey'>


In [27]:
from cryptography.hazmat.primitives.asymmetric import padding
from cryptography.hazmat.primitives import hashes

# шифрование текста при помощи RSA-OAEP (это усиливающая классический RSA cхема с использованием двух криптостойких хеш-функций и паддинга, если интересно, можно почитать здесь https://habr.com/ru/post/99376/)
text= bytes('кто прочитал тот воскреснет', 'UTF-8')
c_text = public_key.encrypt(text, padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))

print(c_text)

b'I\xb2\x88\xf5\x93O\xd2\x0c_\x87\xa0.K\xf4\xa1\xde\xb0K\xb9\x95\xaf\xfeeq\xa3E\xc3\xb5K\xfb\xb7\xa3l\x82\xc2XGNHD4\x9c\xee\xb8\xc3\xed\x17\xd7\xfe\x1e\xe0\xe1\x14M\xe0\r\x06\xc7\x82h\x86\xe1\xbc\x90\x07\xa4D\xda\xdcrQ y-\x1fa\x84K^\xd1\x1c\xa9\xc6T1}h\x07]\x1a\xe7n\xaf\r\xca\x1d\xc7\xc5x\xb5\xca\xb5d\xef\xeawX\xdd\xff\x94s`M\x0e\xe5\xb0w&\xbf\\.\xf6\xf2S\xccCB\xd6\xd6\x14\x9aA\xe1(\x13\x1d\x13\xa5/\xbd\xd4#r\xf6]\x98\xe7\xca\xf7=K!W\x1b4\xa4\xe4\xbfr\x13s\xc4)!\xd5\xbe2\xc0\xafC+=m"S\x0e\xa7\xc3d\xdc\xe1.\xac\x1a\x02\xeae\xfc/\x9c,!\x08\xe3#\xe3\x17\xaf\xcd\xc6L\xb1\xa7\xff\xe6q\xa7\x14\xc8\x19Jq\xb89\xe9\x15\xf0S\xd9\xaeQ\xeeY\x15\x15>\x9fm\xb2\xe6\xad2\xdaT\xf8\xe0O\xf7f\nc\xb2\xd1\xbe\xca\xfaX\xf7_\xca\x90\x1eM\xf2L\xf0'


In [28]:
# дешифрование текста асимметричным алгоритмом
dc_text = private_key.decrypt(c_text,padding.OAEP(mgf=padding.MGF1(algorithm=hashes.SHA256()),algorithm=hashes.SHA256(),label=None))

print(dc_text.decode('UTF-8'))

кто прочитал тот воскреснет


##Варианты заданий
Далее приведены симметричные алгоритмы шифрования для различных вариантов лабораторной. Вариант $v$ предлагается выбрать как $ v = rem(i,9)$, где $i$ - порядковый номер студента в списке.    

Грубо говоря, варианты (в основном) отличаются вызовом определенного метода библиотеки и длиной ключа, который необходимо сгенерировать. 

*Помимо работающего приложения, для сдачи необходимо изучить и рассказать  исторические аспекты используемого симметричного алгоритма - когда, кем и для каких целей он был разработан, считается ли  сегодня криптостойким, к каким атакам алгоритм уязвим, принят ли он в качестве стандарта шифрования на официальном уровне каким-либо государством.*

Варианты:  
1. [**AES**](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.algorithms.AES), длина ключа 128,192,256 бит - *предусмотреть пользовательский выбор длины ключа*;  
2. [**Camellia**](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.algorithms.Camellia), длина ключа 128,192,256 бит - *предусмотреть пользовательский выбор длины ключа*;  
3. [**ChaCha20**](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.algorithms.ChaCha20), длина ключа 256 бит, есть дополнительный параметр - одноразовое случайное число 128 бит - *предусмотреть сериализацию/десериализацию этого параметра*;  
4. [**3DES**](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.algorithms.TripleDES), длина ключа 64, 128, 192 бит - *предусмотреть пользовательский выбор длины ключа*;  
5. [**CAST5**](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.algorithms.CAST5), длина ключа от 40 до 128 бит с шагом 8 бит - *предусмотреть пользовательский выбор длины ключа*;  
6. [**SEED**](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.algorithms.SEED), длина ключа 128 бит;  
7. [**SM4**](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.algorithms.SM4), длина ключа 128 бит;  
8. [**Blowfish**](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.algorithms.Blowfish), длина ключа от 32 до 448 бит с шагом 8 бит - *предусмотреть пользовательский выбор длины ключа*;  
9. [**IDEA**](https://cryptography.io/en/latest/hazmat/primitives/symmetric-encryption/#cryptography.hazmat.primitives.ciphers.algorithms.ARC4), длина ключа 128 бит.  
